<a href="https://colab.research.google.com/github/Sims0212/Sims0212/blob/main/tilnote_whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

노트북을 꼭 복사해서 사용하세요. (파일 - Drive에 사본 저장)\
복사하지 않으면 이 노트북 파일의 세션에 올린 파일이 남을 수 있습니다.

In [ ]:
! pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-odcsfb9c
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-odcsfb9c
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━

[checkpoint] 런타임 - 런타임 유형 변경에서 하드웨어를 GPU가 되어있는지 확인해 주세요.

받아쓰기 할 파일을 올려주세요. (참고로 파일은 좌측 상단의 파일 아이콘의 /content 폴더에 저장됩니다. 😀)

**파일이 클 경우에는 1번 구글 드라이브를 마운트해서 올리는게 훨씬 빠릅니다. 용량이 큰 경우 구글 드라이브를 마운트하는 코드를 로드해서 활용해 주세요.**

테스트나 작은 파일의 경우 2번을 통해 진행해 주세요.

In [ ]:
# 1번
# 파일 용량이 큰 경우에는 구글 드라이브에 업로드한 후 진행해야지 빠름.
# 권한이 필요하다. 마운트된 드라이브는 세션이 고립된 환경에서 실행되기 때문에 다른 세션은 마운트된 폴더를 볼 수 없다.
# 동영상일 경우 음성 파일만 export 하면 된다.
# google drive에 파일을 올린 후
# whisper 부분의 '/content/gdrive/MyDrive/youtube/myfile.mov' 와 같이 경로 적절히 수정.
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
filepath = '/content/gdrive/MyDrive/youtube/myfile.mov'

In [ ]:
# 2번
# 작은 파일의 경우 직접 업로드. 앞의 구글 드라이드 업로드를 했다면 넘어가기.
from google.colab import files
uploaded = files.upload()
file_names = list(uploaded.keys())
print(file_names)
filepath = f'/content/{file_names[0]}'

In [ ]:
#   --task translate 를 지정하면 영어로 번역 자막을 생성함.
! whisper "{filepath}" --language Korean --model large

추가 기능 (선택 기능)\
목소리를 마이크로 입력받아 텍스트로 변환하기\
gradio 웹 인터페이스 활용

In [ ]:
import whisper
model = whisper.load_model("base")

# 음성을 받아서 텍스트로 처리하는 함수
def transcribe(audio):

    # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)

    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # detect the spoken language
    _, probs = model.detect_language(mel)
    print(f"Detected language: {max(probs, key=probs.get)}")

    # decode the audio
    options = whisper.DecodingOptions()
    result = whisper.decode(model, mel, options)
    return result.text

In [ ]:
# gradio 설치 및 런칭
! pip install gradio -q

import gradio as gr
import time

  gr.Interface(
    title = 'OpenAI Whisper ASR Gradio Web UI',
    fn=transcribe,
    inputs=[
        gr.Audio(source="microphone", type="filepath")
    ],
    outputs=[
        "textbox"
    ],
    live=True).launch()